In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
os.environ["TEST"] = "False"
os.chdir(Path.cwd().parent)
import json
import sqlite3

import pandas as pd
import numpy as np
from dotenv import find_dotenv, load_dotenv
from elasticsearch import Elasticsearch

from podology.data.Episode import Episode
from podology.data.EpisodeStore import EpisodeStore
from podology.data.Transcript import Transcript
from podology.search.utils import make_index_name

from config import DB_PATH

# set cwd one level up:
load_dotenv(find_dotenv())

In [ ]:
episode_store = EpisodeStore()
ep = episode_store["2KGeX"]
t = Transcript(ep)

In [ ]:
es_client = Elasticsearch(
    "http://localhost:9200",
    basic_auth=(os.getenv("ELASTIC_USER"), os.getenv("ELASTIC_PASSWORD")),
    # verify_certs=True,
    # ca_certs=basedir / "http_ca.crt"
)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Or your embedding model

eid = "r7Hpj"
term = "trying a new biscuit variety"
search_embedding = model.encode(term).tolist()
search_embedding[:5]

In [ ]:
# Vector similarity search
vector_query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"eid": eid}}
            ]
        }
    },
    "knn": {
        "field": "embedding",
        "query_vector": search_embedding,
        "k": 1000,
        "num_candidates": 1000,
        "filter": {"term": {"eid": eid}}
    },
    "_source": ["eid", "text", "start", "end", "title"],
    "size": 1000
}

response = es_client.search(index="test_chunks", body=vector_query)

chunk_similarities = [
    {
        "start": hit["_source"]["start"],
        "end": hit["_source"]["end"],
        "similarity_score": hit["_score"],
    }
    for hit in response["hits"]["hits"]
]

relevance_df = pd.DataFrame(chunk_similarities).sort_values("start")

In [ ]:
relevance_df

In [ ]:
def bin_relevance_scores(relevance_df, ep_duration, n_bins=500):
    """
    Bin relevance scores into time-based bins, averaging overlapping chunks.
    """
    if relevance_df.empty:
        return pd.DataFrame({'bin_start': [], 'bin_end': [], 'avg_similarity': []})
    
    # Get the total time span
    min_time = 0
    max_time = ep_duration
    
    # Create bin edges
    bin_edges = np.linspace(min_time, max_time, n_bins + 1)
    
    # Calculate bin centers and create result dataframe
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_starts = bin_edges[:-1]
    bin_ends = bin_edges[1:]
    
    binned_scores = []
    
    for i in range(n_bins):
        bin_start = bin_edges[i]
        bin_end = bin_edges[i + 1]
        
        # Find chunks that overlap with this bin
        overlapping_chunks = relevance_df[
            (relevance_df['start'] < bin_end) & (relevance_df['end'] > bin_start)
        ]
        
        if len(overlapping_chunks) > 0:
            # Calculate overlap weights for averaging
            weighted_scores = []
            total_weight = 0
            
            for _, chunk in overlapping_chunks.iterrows():
                # Calculate overlap duration
                overlap_start = max(chunk['start'], bin_start)
                overlap_end = min(chunk['end'], bin_end)
                overlap_duration = overlap_end - overlap_start
                
                if overlap_duration > 0:
                    # Weight by overlap duration
                    weighted_scores.append(chunk['similarity_score'] * overlap_duration)
                    total_weight += overlap_duration
            
            if total_weight > 0:
                avg_score = sum(weighted_scores) / total_weight
            else:
                avg_score = overlapping_chunks['similarity_score'].mean()
        else:
            # No chunks in this bin
            avg_score = 0.0
        
        binned_scores.append(avg_score)
    
    return pd.DataFrame({
        'bin_start': bin_starts,
        'bin_end': bin_ends,
        'bin_center': bin_centers,
        'avg_similarity': binned_scores
    })

binned_relevance = bin_relevance_scores(relevance_df)

In [ ]:
binned_relevance.head(20)

In [ ]:
def debug_chunk_status_issue():
    conn = sqlite3.connect(DB_PATH)
    
    cur = conn.execute("SELECT * FROM episodes LIMIT 0")
    actual_columns = [desc[0] for desc in cur.description]
    print(f"  Columns: {actual_columns}")

debug_chunk_status_issue()

In [ ]:
conn = sqlite3.connect(DB_PATH)
cur = conn.execute("SELECT * FROM episodes")
cur.fetchall()

In [ ]:
# migration_script.py
import sqlite3
from config import DB_PATH

def migrate_duration_column():
    conn = sqlite3.connect(DB_PATH)
    
    try:
        # Check current schema
        cur = conn.execute("PRAGMA table_info(episodes)")
        columns = cur.fetchall()
        print("Current schema:")
        for col in columns:
            print(f"  {col[1]}: {col[2]}")
        
        # Perform migration
        print("Starting migration...")
        
        conn.execute("BEGIN TRANSACTION")
        
        # Your migration code here (use Method 2 approach)
        conn.execute("""
            CREATE TABLE episodes_new (
                eid TEXT PRIMARY KEY,
                url TEXT UNIQUE,
                title TEXT,
                pub_date TEXT,
                description TEXT,
                duration FLOAT,
                transcript_status TEXT,
                transcript_wcstatus TEXT,
                audio_status TEXT,
                chunk_status TEXT
            )
        """)
        
        conn.execute("""
            INSERT INTO episodes_new 
            SELECT 
                eid, url, title, pub_date, description,
                CAST(duration AS FLOAT) as duration,
                transcript_status, transcript_wcstatus, 
                audio_status, chunk_status
            FROM episodes
        """)
        
        conn.execute("DROP TABLE episodes")
        conn.execute("ALTER TABLE episodes_new RENAME TO episodes")
        
        conn.execute("COMMIT")
        print("Migration completed successfully!")
        
    except Exception as e:
        conn.execute("ROLLBACK")
        print(f"Migration failed: {e}")
        raise
    finally:
        conn.close()

if __name__ == "__main__":
    migrate_duration_column()

In [ ]:
import os

def nuclear_option_rebuild_database():
    """Completely rebuild the database from scratch"""
    import shutil
    from datetime import datetime
    
    # Backup the current database
    backup_path = f"{DB_PATH}.backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    shutil.copy2(DB_PATH, backup_path)
    print(f"Backed up database to: {backup_path}")
    
    # Get all the data first
    conn = sqlite3.connect(DB_PATH)
    cur = conn.execute("SELECT * FROM episodes")
    all_data = cur.fetchall()
    
    # Get just the columns that work
    cur = conn.execute("SELECT eid, url, title, pub_date, description, duration, transcript_status, transcript_wcstatus, audio_status FROM episodes")
    working_data = cur.fetchall()
    conn.close()
    
    print(f"Extracted {len(working_data)} rows")
    
    # Delete the database file
    os.remove(DB_PATH)
    print(f"Deleted {DB_PATH}")
    
    # Create fresh database
    conn = sqlite3.connect(DB_PATH)
    
    # Create table with correct schema
    conn.execute("""
        CREATE TABLE episodes (
            eid TEXT PRIMARY KEY,
            url TEXT UNIQUE,
            title TEXT,
            pub_date TEXT,
            description TEXT,
            duration FLOAT,
            transcript_status TEXT,
            transcript_wcstatus TEXT,
            audio_status TEXT,
            chunk_status TEXT
        )
    """)
    
    # Insert all data
    for row in working_data:
        eid, url, title, pub_date, description, duration, transcript_status, transcript_wcstatus, audio_status = row
        
        # Convert duration
        try:
            duration_float = float(duration) if duration is not None else None
        except:
            duration_float = None
            
        conn.execute("""
            INSERT INTO episodes 
            (eid, url, title, pub_date, description, duration, transcript_status, transcript_wcstatus, audio_status, chunk_status)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (eid, url, title, pub_date, description, duration_float, transcript_status, transcript_wcstatus, audio_status, None))
    
    conn.commit()
    conn.close()
    
    print("Database rebuilt successfully!")
    
    # Test the new database
    conn = sqlite3.connect(DB_PATH)
    cur = conn.execute("PRAGMA table_info(episodes)")
    columns = cur.fetchall()
    print("Final schema:")
    for col in columns:
        print(f"  {col[1]}: {col[2]}")
    conn.close()

nuclear_option_rebuild_database()  # Uncomment to run